In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root is set:", project_root)


Project root is set: /home/homeflo/workspace/project_trading/Trading-Bot


#### === Step 1: Load data  ===

In [287]:
symbol =  "GBP_USD_M1_09-01-2025_10-01-2025_with_volume.pkl" #"GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl"  #"XAUUSD_H1.csv"
# symbol =  "GBP_USD_M1_08-01-2025_09-01-2025_with_volume.pkl"
print(f"📊 Loading data for {symbol} with days_back=60...")
ltf1 = "1M"
ltf2 = "2M"
mtf = "15M"
htf = "4H"
# Use the same data loading function
from core.data_loader import load_and_resample
resampled = load_and_resample(f"../data/{symbol}", days_back=60)
ltf1_data = resampled.get(ltf1)
ltf2_data = resampled.get(ltf2)
mtf_data = resampled.get(mtf)
htf_data = resampled.get(htf)

if ltf1_data is None or ltf1_data.empty:
    print(f"❌ ERROR: No data loaded for the '{ltf1}' timeframe.")
elif ltf2_data is None or ltf2_data.empty:
    print(f"❌ ERROR: No data loaded for the '{ltf2}' timeframe.")
elif mtf_data is None or mtf_data.empty:
    print(f"❌ ERROR: No data loaded for the '{mtf}' timeframe.")
elif htf_data is None or htf_data.empty:
    print(f"❌ ERROR: No data loaded for the '{htf}' timeframe.")
    
else:
    print(f"📊 Loaded {len(ltf1_data):5} '{ltf1}' candles from \t{ltf1_data.index.min()} to {ltf1_data.index.max()}")
    print(f"📊 Loaded {len(ltf2_data):5} '{ltf2}' candles from \t{ltf2_data.index.min()} to {ltf2_data.index.max()}")
    print(f"📊 Loaded {len(mtf_data):5} '{mtf}' candles from \t{mtf_data.index.min()} to {mtf_data.index.max()}")
    print(f"📊 Loaded {len(htf_data):5} '{htf}' candles from \t{htf_data.index.min()} to {htf_data.index.max()}")
    data = {ltf1: ltf1_data, ltf2: ltf2_data, mtf: mtf_data, htf: htf_data}

📊 Loading data for GBP_USD_M1_09-01-2025_10-01-2025_with_volume.pkl with days_back=60...
📊 Loaded 31462 '1M' candles from 	2025-09-01 00:00:00+00:00 to 2025-10-01 00:00:00+00:00
📊 Loaded 15695 '2M' candles from 	2025-09-01 00:00:00+00:00 to 2025-09-30 23:58:00+00:00
📊 Loaded  2050 '15M' candles from 	2025-09-01 00:00:00+00:00 to 2025-09-30 23:45:00+00:00
📊 Loaded   108 '4H' candles from 	2025-09-01 00:00:00+00:00 to 2025-09-30 16:00:00+00:00


#### === Step 2: Analyse market structure ===

In [288]:
print("🔍 Analyzing market structure...")
from core.structure_builder import analyse_market_structure
from time import perf_counter_ns

t = perf_counter_ns()
swing_high_low_test, trend_test = analyse_market_structure(mtf_data, based_on_atr=True)#dropna()
swing_high_low_test = swing_high_low_test.dropna()
print(f"⏱️  Market structure analysis with ATR took \t {(perf_counter_ns() - t)/ 1_000_000} ms")

t = perf_counter_ns()
swing_high_low_test_2, trend_test_2 = analyse_market_structure(mtf_data, based_on_atr=False)#.dropna()
swing_high_low_test_2 = swing_high_low_test_2.dropna()

print(f"⏱️  Market structure analysis without ATR took \t {(perf_counter_ns() - t) / 1_000_000} ms")

🔍 Analyzing market structure...
⏱️  Market structure analysis with ATR took 	 13.009824 ms
⏱️  Market structure analysis without ATR took 	 54.678529 ms


In [289]:
print("🔍 Analyzing market structure...")
from core.structure_builder import analyse_market_structure

prominence_factor ={
    ltf1: 10,
    ltf2: 9,
    mtf: 4,
    htf: 0.5
}

structure = {}
trends = {}
for tf, df in data.items():
    analysis, trend = analyse_market_structure(df, swing_length=10, trend_window=4, prominence_factor=prominence_factor[tf])
    structure[tf] = analysis.dropna()
    trends[tf] = trend
    print(f"📊 {tf:3}: Total structure points: {len(structure[tf].dropna()):5}, Trend: {trend}")

🔍 Analyzing market structure...
📊 1M : Total structure points:   148, Trend: sideways
📊 2M : Total structure points:    71, Trend: sideways
📊 15M: Total structure points:    23, Trend: downtrend
📊 4H : Total structure points:    14, Trend: downtrend


In [4]:
from importlib import reload
from core import structure_builder
reload(structure_builder)

<module 'core.structure_builder' from '/home/homeflo/workspace/project_trading/Trading-Bot/core/structure_builder.py'>

In [290]:
structure_mtf = structure[mtf]


#### === Step 3: Detect market events ===

In [291]:
print("🎯 Detecting market events...")
from core.smart_money_concepts import MarketStructureAnalyzer
analyzer = MarketStructureAnalyzer(confidence_threshold=0.5)

all_events = {}
for tf, df in data.items():
    print(f"🔍 Analyzing {tf} timeframe...")
    events = analyzer.analyse_market_events(structure[tf])
    print(f"📊 {tf:3}: Detected events: {len(events):5}\n")
    all_events[tf] = events


🎯 Detecting market events...
🔍 Analyzing 1M timeframe...
Index 2: SwingType.HL @ 1.33815 - Trend before: sideways
📊 1M : Detected events:   148

🔍 Analyzing 2M timeframe...
Index 2: SwingType.LH @ 1.34188 - Trend before: sideways
📊 2M : Detected events:    71

🔍 Analyzing 15M timeframe...
Index 2: SwingType.HL @ 1.34166 - Trend before: sideways
📊 15M: Detected events:    23

🔍 Analyzing 4H timeframe...
Index 2: SwingType.HL @ 1.34835 - Trend before: sideways
📊 4H : Detected events:    14



In [275]:
from importlib import reload
from core import smart_money_concepts
reload(smart_money_concepts)

<module 'core.smart_money_concepts' from '/home/homeflo/workspace/project_trading/Trading-Bot/core/smart_money_concepts.py'>

In [89]:
from enum import Enum
from dataclasses import dataclass
from typing import Optional
import pandas as pd

class SwingType(Enum):
    HH = "HH"
    HL = "HL"
    LH = "LH"
    LL = "LL"

@dataclass
class StructurePoint:
    timestamp: pd.Timestamp
    price: float
    swing_type: SwingType
    retracement: Optional[float] = None  # Optional retracement value

swing = StructurePoint(
    timestamp=pd.Timestamp("2024-01-01 00:00:00"),
    price=1.2345,
    swing_type=SwingType.HH,
    retracement=0.618)

swing.swing_type == SwingType.HH


True

In [292]:
mtf_events = all_events[mtf]

#### === Step 4: Initialize and run backtrader ===

#### === Step 5: Run backtest ===

#### === Step 6: Display results ===

#### === Step 7: Generate visualizations ===

In [393]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(mtf_data, swing_high_low_test_2.dropna(), "GBPUSD", mtf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [278]:
from importlib import reload
from utils import structure_trend_plotter
reload(structure_trend_plotter)

<module 'utils.structure_trend_plotter' from '/home/homeflo/workspace/project_trading/Trading-Bot/utils/structure_trend_plotter.py'>

In [293]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(mtf_data, structure[mtf].dropna(), mtf_events, "GBPUSD", mtf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
<class 'list'>
205
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [100]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(htf_data, structure[htf].dropna(), "GBPUSD", htf)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
10
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [ ]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(ltf1_data, structure[ltf1].dropna(), "GBPUSD", ltf1)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [105]:
from utils.structure_trend_plotter import StructureTrendPlotter
import plotly.io as pio
pio.renderers.default = "browser"

plotter = StructureTrendPlotter()

try:
    # Test Chart: Trend Detection
    print("\n-> Testing Chart : Trend Detection")
    fig1 = plotter.plot_structure(ltf2_data, structure[ltf2].dropna(), "GBPUSD", ltf2)
    print("✅ Chart 1 created successfully")
    fig1.show()
    
    print("\n🎉 All charts generated successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


-> Testing Chart : Trend Detection
1569
✅ Chart 1 created successfully

🎉 All charts generated successfully!


In [195]:
from importlib import reload
from utils import structure_trend_plotter
reload(structure_trend_plotter)

<module 'utils.structure_trend_plotter' from '/home/homeflo/workspace/project_trading/Trading-Bot/utils/structure_trend_plotter.py'>